<html><body><H2><u>Implementation of Deep Canonical Correlation AutoEncoders (DCCAE)- Documented
</H2><br></body></html>

<html><body><pre>
<h3>We have used the following libraries from Pytorch:</h3>
           <b>torch</b>: Tensor computation (like NumPy) and has features for Deep neural networks
           <b>nn</b>: These are Neural networks implemented in torch
           <b>optim</b>: For implementing various optimization algorithms
           <b>functional</b>: Applies a 1D convolution over an input signal composed of several input planes</pre><br></body></html>

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

from DCCAE_repo.configuration import Config

#### <br>Here we have defined 'create_encoder'  and 'create_decoder' functions with arguments as mentioned in the config file (Parameters of the neural network).
This function creates and returns the encoder for the multidimentional training data set<br>

In [ ]:
def create_encoder(config, i):
    encoder = config.encoder_models[i](config.hidden_layer_sizes[i], config.input_sizes[i], config.latent_dims).double()
    return encoder


In [ ]:
def create_decoder(config, i):
    decoder = config.decoder_models[i](config.hidden_layer_sizes[i], config.latent_dims, config.input_sizes[i]).double()
    return decoder

<br>
<b>Description of the DCCAE class and Functions:</b>
<pre>We have created a class DCCAE to contain our multiviewed vectors (neural networks), the class
contains the following functions:
This class is the sub class of nn.Module parent class<br>
<b>__init__():</b> constructor, Here the parameters for the neural network are initialized as mentioned in the config file
<b>encode()</b>: Here the data for the neural networks is provided. This returns a tuple of the value and its enum
<b>forward()</b>: Here the data is forwarded to the encoder
<b>decode()</b>: Here the data for decoding is provided in recon array. This too returns a tuple like the encode function
<b>update_weights()</b>: This function assigns wights to the data in the nn, we have used optimizer to optimize from a vector    of parameters.
<b>loss()</b>: Here the net loss from the nn is calculated
<b>recon_loss()</b>: Here the losses are reconstructed


In [ ]:
class DCCAE(nn.Module):

    def __init__(self, config: Config = Config):
        super(DCCAE, self).__init__()
        views = len(config.encoder_models)
        self.encoders = torch.nn.ModuleList([create_encoder(config, i) for i in range(views)])
        self.decoders = torch.nn.ModuleList([create_decoder(config, i) for i in range(views)])
        self.lam = config.lam
        self.objective = config.objective(config.latent_dims)
        self.optimizers = [optim.Adam(list(self.encoders[i].parameters()) + list(self.decoders[i].parameters()),
                                      lr=config.learning_rate) for i in range(views)]
        


    def encode(self, *args):
        z = []
        for i, arg in enumerate(args):
            z.append(self.encoders[i](arg))
        return tuple(z)

    def forward(self, *args):
        z = self.encode(*args)
        return z

    def decode(self, *args):
        recon = []
        for i, arg in enumerate(args):
            recon.append(self.decoders[i](arg))
        return tuple(recon)

    def update_weights(self, *args):
        [optimizer.zero_grad() for optimizer in self.optimizers]
        loss = self.loss(*args)
        loss.backward()
        [optimizer.step() for optimizer in self.optimizers]
        return loss

    def loss(self, *args):
        z = self.encode(*args)
        recon = self.decode(*z)
        recon_loss = self.recon_loss(args, recon)
        return self.lam * recon_loss + self.objective.loss(*z)

    @staticmethod
    def recon_loss(x, recon):
        recons = [F.mse_loss(recon[i], x[i], reduction='sum') for i in range(len(x))]
        return torch.stack(recons).sum(dim=0)
